In [1]:
import icechunk
import zarr

Lets create an in-memory icechunk store

In [2]:
store = await icechunk.IcechunkStore.create(
    storage=icechunk.StorageConfig.memory(""), mode="w"
)
store

Ok! Lets create some data!

In [4]:
group = zarr.group(store=store, overwrite=True)
group

Group(_async_group=<AsyncGroup <icechunk.IcechunkStore object at 0x10c191110>>)

In [5]:
air_temp = group.create_array(
    "air_temp", shape=(1000, 1000), chunk_shape=(100, 100), dtype="i4"
)
air_temp

<Array <icechunk.IcechunkStore object at 0x10c191110>/air_temp shape=(1000, 1000) dtype=int32>

In [6]:
async for key in store.list():
    print(key)

zarr.json
air_temp/zarr.json


In [7]:
air_temp[:, :] = 42

In [8]:
air_temp[200, 6]

array(42, dtype=int32)

Now that we have set the values, lets commit

In [10]:
snapshot_id = await store.commit("Initial commit")
snapshot_id

'SVSF6Y1CR9DX61D4MHFX6WB4JG'

Okay now we can change the data

In [11]:
air_temp[:, :] = 54

In [12]:
air_temp[200, 6]

array(54, dtype=int32)

And we can commit again

In [13]:
new_snapshot_id = await store.commit("Change air temp to 54")
new_snapshot_id

'XGSEWCBDBRT4VE69BXC2JPV9MC'

Cool, now lets checkout the original snapshot and see if the value is 42 again

In [14]:
await store.checkout(snapshot_id=snapshot_id)
air_temp[200, 6]

array(42, dtype=int32)